In [1]:
import os

# Membaca data yang sudah dibersihkan dari karakter yang tidak diperlukan serta dari daftar kata tak baku dan stop words
# Pastikan untuk mengganti path dengan absolute path direktorimu jika baris berikut dijalankan ulang, atau restart kernel.
os.chdir("output")
base = "prastyo-sentiment_posneg-clean-slang-stop.txt"

### We'll remove duplicate lines first

In [2]:
from collections import OrderedDict


input_stream = open(base, "r", encoding="utf8")
text = input_stream.readlines()
input_stream.close()
print("before:", len(text))

# menghapus duplikasi kalimat dengan mengonversinya ke 'ordered dictionary'
newtext = list(OrderedDict.fromkeys(text))
print("after:", len(newtext))

before: 1918
after: 1660


In [3]:
print(text[:2],)
print(newtext[:2])

['ya utang pemerintah utang bangsa indonesia hutang nya captain covid kalo orang terserah but ya apapun pemerintah bala tentara allah swt sahabat nya \tneg\n', 'yuk kawal kebijakan pemerintah disalah oknum bertanggung indonesia menang lawan covid \tpos\n']
['ya utang pemerintah utang bangsa indonesia hutang nya captain covid kalo orang terserah but ya apapun pemerintah bala tentara allah swt sahabat nya \tneg\n', 'yuk kawal kebijakan pemerintah disalah oknum bertanggung indonesia menang lawan covid \tpos\n']


In [4]:
# Menyimpan teks yang sudah dibersihkan ke file
output = os.path.splitext(base)[0]+'-dup.txt'
with open(output, 'w') as f:
    for line in newtext:
        f.write(str(line))

In [5]:
import pandas as pd

# Kita akan menggunakan kolom teks saja di tahap ini, tanpa kolom label
Corpus = pd.read_csv(output, encoding='latin-1', header=None, sep="\t", names=['text', 'label'], usecols=['text'], dtype=str)

# # ALTERNATIF
# # Memisahkan teks dan label dengan membuat DataFrame dari list 'ordered dict'
# df = pd.DataFrame(newtext, columns=['text'])
# df[['text','label']] = df['text'].str.split('\t',expand=True)
# df['label'] = df['label'].str.split('\n',expand=True)

## Pelabelan dengan **InSet** dan **sentiwords_id** (dari sentistrength_id)

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib

# Memanfaatkan nltk VADER untuk menggunakan leksikon kustom
sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
# membersihkan leksikon VADER default
sia1A.lexicon.clear()
sia1B.lexicon.clear()
sia2.lexicon.clear()

# Membaca leksikon InSet
# Leksikon InSet lexicon dibagi menjadi dua, yakni polaritas negatif dan polaritas positif;
# kita akan menggunakan nilai compound saja untuk memberi label pada suatu kalimat
with open('../leksikon/inset/_json_inset-neg.txt') as f:
    data1A = f.read()
with open('../leksikon/inset/_json_inset-pos.txt') as f:
    data1B = f.read()

# Membaca leksikon sentiwords_id
with open('../leksikon/sentistrength_id/_json_sentiwords_id.txt') as f:
    data2 = f.read()

# Mengubah leksikon sebagai dictionary
insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)
senti = json.loads(data2)

# Update leksikon VADER yang sudah 'dimodifikasi'
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)
sia2.lexicon.update(senti)

print(reprlib.repr(sia1A.lexicon))
print(reprlib.repr(sia1B.lexicon))
print(reprlib.repr(sia2.lexicon))

{'(barang) bekas': -4, '(olahraga) bokser': -5, '(tua) uzur': -3, 'Anda': -4, ...}
{'(hujan) gerimis': 1, '(warna) dadu': 3, 'Ahad': 3, 'Sri paduka': 4, ...}
{'abadi': 5, 'absen': -3, 'abu-abu': -1, 'acuh': 4, ...}


### Contoh: menghitung skor polaritas dari suatu kalimat

In [7]:
sample = "kalau kamu sudah sampai sini kamu hebat ayo terus kamu pasti bisa"
print("insetNeg: ", sia1A.polarity_scores(sample))
print("insetPos: ", sia1B.polarity_scores(sample))
print("insetCpdSum: 'compound':", sia1A.polarity_scores(sample)["compound"] + sia1B.polarity_scores(sample)["compound"])

print("senti\t: ", sia2.polarity_scores(sample))

insetNeg:  {'neg': 0.526, 'neu': 0.474, 'pos': 0.0, 'compound': -0.875}
insetPos:  {'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.9517}
insetCpdSum: 'compound': 0.07669999999999999
senti	:  {'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.6124}


### Fungsi untuk mengklasifikasikan kalimat sebagai negatif/positif berdasarkan nilai *compound*

In [8]:
def is_positive_inset(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

def is_positive_senti(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia2.polarity_scores(tweet)["compound"] > 0

In [9]:
tweets = Corpus["text"]

# Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari insetNeg dan insetPos
output = os.path.splitext(base)[0]+'-lb-inset.txt'
with open(output, 'w') as f:
    for tweet in tweets:
        if is_positive_inset(tweet) == True:
            label = "pos"
        else:
            label = "neg"
        f.write(str(label+'\n'))

# Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari sentiwords_id
output = os.path.splitext(base)[0]+'-lb-senti.txt'
with open(output, 'w') as f:
    for tweet in tweets:
        if is_positive_senti(tweet) == True:
            label = "pos"
        else:
            label = "neg"
        f.write(str(label+'\n'))